<a href="https://colab.research.google.com/github/napoles-uach/ML-FCQ/blob/main/keras_numeros_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q

In [ ]:
pip install streamlit_drawable_canvas

In [7]:
%%writefile app.py
import streamlit as st
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



from tensorflow.keras.datasets import mnist

from streamlit_drawable_canvas import st_canvas

st.title('Ejemplo Keras/MNIST')
col1,col2=st.columns([4,4])
with col1:
  st.image('https://miro.medium.com/max/1188/1*Ft2rLuO82eItlvJn5HOi9A.png')

with col2:
  st.image('https://miro.medium.com/max/1160/1*T-AyPeCdLDzHI0R952EmYg.gif')

def convim(im):
  lista1=[]
  for i in range(28):
    lista2=[]
    for j in range(28):
      bit=np.sum(im[i][j][:])
      lista2.append(bit)
    lista1.append(lista2)
  return lista1


(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

images_test = test_images


model = keras.Sequential([
 layers.Dense(512, activation="relu"),
 layers.Dense(10, activation="softmax")
])
st.subheader('Definición de la arquitectura de la red neuronal :+1:')



code = '''model = keras.Sequential([
 layers.Dense(512, activation="relu"),
 layers.Dense(10, activation="softmax")
])'''

st.code(code, language='python')


st.subheader('optimización usando backpropagation')
model.compile(optimizer="rmsprop",
 loss="sparse_categorical_crossentropy",
 metrics=["accuracy"])
code_comp = '''model.compile(optimizer="rmsprop",
 loss="sparse_categorical_crossentropy",
 metrics=["accuracy"])
'''
st.code(code_comp,language='python')

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255
test_images[0].shape
#model.fit(train_images, train_labels, epochs=5, batch_size=128)
#@st.experimental_singleton
def model_fit(model):
  history = model.fit(train_images, train_labels, epochs=5, batch_size=128)
  return history
code_fit = 'model.fit(train_images, train_labels, epochs=5, batch_size=128)'
st.code(code_fit,language='python')

train = st.checkbox('train')
if train:
  history = model_fit(model)
  df = pd.DataFrame(history.history)
  st.line_chart(df)




test_digits = test_images[0:10]
predictions = model.predict(test_digits)
n=st.sidebar.selectbox("Indice",[0,1,2,3,4,5,6,7,8,9])
fig, ax = plt.subplots()
im = ax.imshow(images_test[n], cmap="binary")
st.sidebar.pyplot(fig)
st.sidebar.write(images_test[n])

st.sidebar.header("Prediccion: "+str(predictions[n].argmax()))
st.sidebar.write("Valor verdadero "+str(test_labels[n]))

test_loss, test_acc = model.evaluate(test_images, test_labels)
st.write(f"test_acc: {test_acc}")

canvas_result = st_canvas(stroke_width= 2,height=28,width=28)
ima =convim(canvas_result.image_data)
ima=np.array(ima)
ima = ima.reshape( 28 * 28)
ima = ima.astype("float32") / 255
ima=[ima]
ima=np.array(ima)
pred_ima=model.predict(ima)
st.write(pred_ima.argmax())

Overwriting app.py


In [6]:
!streamlit run app.py & npx localtunnel --port 8501

2022-10-11 18:56:48.244 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.547s



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.85.143.246:8501

your url is: https://blue-ideas-draw-34-85-143-246.loca.lt
2022-10-11 18:57:03.081026: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
313/313 [==============================] - 1s 2ms/step - loss: 2.3645 - accuracy: 0.0862
2022-10-11 18:57:04.441 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 562, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 101, in <module>
    ima =convim(canvas_result.image_data)
  File "/content/app.py", line 26, in convim
    bit=np.sum(im[i][j][:])
TypeError: 'NoneType' object is not subscri